In [24]:
# ALWAYS RUN THIS CELL
# widen jupyter notebook window
from IPython.display import display, HTML
display(HTML("<style>.container {width:95% !important; }</style>"))

from IPython.display import display, HTML
display(HTML("<style>:root { --jp-notebook-max-width: 100% !important; }</style>"))

This requires having `bnpm`. Get it with `pip install bnpm` or here: https://github.com/RichieHakim/basic_neural_processing_modules

In [2]:
%load_ext autoreload
%autoreload 2
from bnpm import server

In [3]:
import os
from pathlib import Path
import functools
import tempfile
import getpass
import gc

In [4]:
import numpy as np
import natsort

------------
# ==== Start ====

## Prepare username + password

In [5]:
remote_host_transfer = "transfer.rc.hms.harvard.edu"
remote_host_compute = "o2.hms.harvard.edu"
username = input('Username: ')

use_localSshKey = False

pw = server.pw_encode(getpass.getpass(prompt='Password: ')) if use_localSshKey==False else None

path_sshKey = '/home/rich/.ssh/id_rsa' if use_localSshKey else None

Username:  rh183
Password:  ········


## Prepare `sftp` and `ssh` objects

In [6]:
## initialize ssh_transfer
ssh_t = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)

In [7]:
ssh_t.o2_connect(
    hostname=remote_host_transfer,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [8]:
## initialize ssh_compute
ssh_c = server.ssh_interface(
    nbytes_toReceive=20000,
    recv_timeout=1,
    verbose=True,
)
ssh_c.o2_connect(
    hostname=remote_host_compute,
    username=username,
    password=server.pw_decode(pw),
    key_filename=path_sshKey,
    look_for_keys=False,
    passcode_method=1,
    verbose=0,
    skip_passcode=False,    
)

In [9]:
sftp = server.sftp_interface(ssh_client=ssh_t.client)

In [10]:
prompt_snip = f'[{username}'

In [11]:
send_expect = functools.partial(ssh_c.send_expect, str_success=prompt_snip)

# Batch run

In [15]:
paths_found = natsort.natsorted(sftp.search_recursive(
    path='/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/',
    search_pattern_re='exp',
    max_depth=5,
    find_folders=True,
    find_files=False,
    verbose=True,
))

/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/20230531/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/20230523/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/20230517/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/20230515/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/20230430/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/20230528/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/20230601/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/20230512/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/20230510/scanimage_data/exp
/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/20230511/scanimage_data/exp
/n/data1/h

In [18]:
dates_toUse = natsort.natsorted([Path(p).parts[-3] for p in paths_found])
dates_toUse

['20230430',
 '20230501',
 '20230502',
 '20230503',
 '20230504',
 '20230505',
 '20230506',
 '20230507',
 '20230508',
 '20230509',
 '20230510',
 '20230511',
 '20230512',
 '20230513',
 '20230514',
 '20230515',
 '20230516',
 '20230517',
 '20230518',
 '20230520',
 '20230521',
 '20230522',
 '20230523',
 '20230524',
 '20230525',
 '20230526',
 '20230527',
 '20230528',
 '20230529',
 '20230530',
 '20230531',
 '20230601']

## Start a small interactive node

In [19]:
send_expect(f'srun --pty -p interactive -t 0-00:30:00 --mem=4G /bin/bash');

bash --pty -p interactive -t 0-00:30:00 --mem=4G /bin/ 

srun: job 14015582 queued and waiting for resources



## Pull/update repo

In [ ]:
send_expect('cd /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP')
send_expect('git pull');

## Activate environment

In [22]:
name_env = '/n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'

send_expect(f'conda activate {name_env}');

ini/rich/virtual_envs/FRhms/neurobio/sabat 

AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-a-16-169 NBA


## Prepare commands

In [ ]:
sftp://rh183@transfer.rc.hms.harvard.edu/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output/mouse_0322N/20230430/scanimage_data/exp/jobNum_0/suite2p

In [75]:
# for date in dates_toUse:
for date in ['20230430']:

    mouse = 'mouse_0322N'
    # date = '20230501'

    name_slurm = mouse + '_' + date[-4:]

    dir_s2p       = str(Path('/n/data1/hms/neurobio/sabatini/gyu/analysis/suite2p_output') / mouse / date / 'scanimage_data' / 'exp' / 'jobNum_0' / 'suite2p' / 'plane0' )

    dir_saveOutputs = str(Path('/n/data1/hms/neurobio/sabatini/rich/analysis/make_dFoF_and_tqm') / mouse / date )


    name_job = 'jobNum_'


    path_script_remote      = str(Path(r'/n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py'))  ## path to a copy of the script (.py) file to run within the repo on the server

    path_dispatcher_local   = str(Path(r'/media/rich/Home_Linux_partition/github_repos/NBAP/behavioral_drift_analysis/').resolve() / 'dispatcher.py')  ## path to the dispatcher.py file on local computer
    path_dispatcher_remote  = (Path(dir_saveOutputs) / 'dispatcher.py').as_posix()  ## path to where you want the dispatcher.py file to be copied onto the server

    ## Expectation for dispatch args: 
    # path_script, dir_save, dir_videos, dir_ROIs, name_job, name_slurm, name_env


    commands = {
    #     'make_dir': f"mkdir -p {dir_data_remote}",
    #     'copy': f"cp -r {dir_s2p_MICROSCOPE} {dir_data_remote}",
    #     'dispatch': f"python {str(path_dispatcher_remote)} {dir_saveOutputs} {path_script_remote} {name_job} {name_slurm} {dir_videos}",  ## dispatcher expecting these args as inputs
    'dispatch': f"\
    python3 \
    {path_dispatcher_remote} \
    {path_script_remote} \
    {dir_saveOutputs} \
    {dir_s2p} \
    {name_job} \
    {name_slurm} \
    {name_env}",
    }
    display(commands)
    
    sftp.mkdir_p(str(Path(path_dispatcher_remote).parent))
    sftp.sftp.put(str(path_dispatcher_local), str(path_dispatcher_remote));
    
    send_expect(commands['dispatch']);

{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230503/aligned_TCA/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230503/aligned_TCA     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/jobNum_0     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230503/jobNum_0     jobNum_     mouse_0322N_0503     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

io/sabatini/rich/github_repos/NBAP/behavio/dispatcher.py     /n/data1/hms/neurob 
rtual_envs/FRum_     mouse_0322N_0503     /n/data1/hms/neurobio/sabatini/rich/vi 

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py to /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230503/aligned_TCA/1_make_consistent_fr_factors.py


Submitting job: jobNum_ 0

Submitted batch job 12693890

AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-a-16-161 NBA


{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230505/aligned_TCA/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230505/aligned_TCA     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/jobNum_0     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230505/jobNum_0     jobNum_     mouse_0322N_0505     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

    pytho
rtual_envs/FRum_     mouse_0322N_0505     /n/data1/hms/neurobio/sabatini/rich/vi 

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py to /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230505/aligned_TCA/1_make_consistent_fr_factors.py
Submitting job: jobNum_ 0

Submitted batch job 12693891

AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-a-16-161 NBA


{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230506/aligned_TCA/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230506/aligned_TCA     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/jobNum_0     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230506/jobNum_0     jobNum_     mouse_0322N_0506     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

...

    p
rtual_envs/FRum_     mouse_0322N_0525     /n/data1/hms/neurobio/sabatini/rich/vi 

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py to /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230525/aligned_TCA/1_make_consistent_fr_factors.py

Submitting job: jobNum_ 0

Submitted batch job 12693899

AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-a-16-161 NBA


{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230526/aligned_TCA/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230526/aligned_TCA     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/jobNum_0     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230526/jobNum_0     jobNum_     mouse_0322N_0526     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

 
rtual_envs/FRum_     mouse_0322N_0526     /n/data1/hms/neurobio/sabatini/rich/vi 

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py to /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230526/aligned_TCA/1_make_consistent_fr_factors.py

Submitting job: jobNum_ 0

Submitted batch job 12693901

AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-a-16-161 NBA


{'dispatch': '    python3     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230530/aligned_TCA/dispatcher.py     /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230530/aligned_TCA     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/jobNum_0     /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230530/jobNum_0     jobNum_     mouse_0322N_0530     /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR'}

 
rtual_envs/FRum_     mouse_0322N_0530     /n/data1/hms/neurobio/sabatini/rich/vi 

dispatcher environment: /n/data1/hms/neurobio/sabatini/rich/virtual_envs/FR

Copying /n/data1/hms/neurobio/sabatini/rich/github_repos/NBAP/behavioral_drift_analysis/1_make_consistent_fr_factors.py to /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230530/aligned_TCA/1_make_consistent_fr_factors.py

Submitting job: jobNum_ 0

Submitted batch job 12693902

AP]$ ata1/hms/neurobio/sabatini/rich/virtual_envs/FR) [rh183@compute-a-16-161 NBA


### check on job

### Check if it worked

In [76]:
paths_found = natsort.natsorted(sftp.search_recursive(
    path='/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701',
    search_pattern_re='tca_refit.h5',
    max_depth=5,
    find_folders=False,
    find_files=True,
))
paths_found

/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230531/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230523/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230517/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230515/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230430/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230528/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230601/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230512/aligned_TCA/jobNum_0/tca_refit.h5
/n/data1

### Download results

In [83]:
dates = [Path(p).parts[-4] for p in paths_found]

In [85]:
paths_download = {d: p for d,p in zip(dates, paths_found)}

In [87]:
directory_save = r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/tca_refit'

In [ ]:
[sftp.get(
    remotepath=p,
    localpath=str(Path(directory_save) / d / 'tca_refit.h5'),
    mkdirs=True,
) for d, p in paths_download.items()];

In [96]:
sftp.get_dir(
    source=r'/n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/',
    target=r'/media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/',
)

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230531/parameters_batch.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230531/parameters_batch.json
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230531/jobNum_0/params.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230531/jobNum_0/params.json
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230531/jobNum_0/config.yaml   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230531/jobNum_0/config.yaml
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230531/jobNum_0/print_log_12089921.log   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230531/jobNum_0/print_log_12089921.log
downloading /n/data1/hms/neurobio/sabatini/rich/analysis

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230523/jobNum_0/analysis_files/VQT_Analyzer.h5   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230523/jobNum_0/analysis_files/VQT_Analyzer.h5
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230523/jobNum_0/analysis_files/Dataset_videos.h5   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230523/jobNum_0/analysis_files/Dataset_videos.h5
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230523/jobNum_0/pipeline_basic.py   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230523/jobNum_0/pipeline_basic.py
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230523/jobNum_0/run_info.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230523/jobNum_

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230517/jobNum_0/run_info.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230517/jobNum_0/run_info.json
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230517/jobNum_0/sbatch_config.sh   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230517/jobNum_0/sbatch_config.sh
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230517/aligned_TCA/parameters_batch.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230517/aligned_TCA/parameters_batch.json
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230517/aligned_TCA/jobNum_0/tca_refit.h5   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230517/aligned_TCA/jobNum_0/tca_refit.h5
downloading /n

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230515/aligned_TCA/jobNum_0/print_log_12685323.log   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230515/aligned_TCA/jobNum_0/print_log_12685323.log
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230515/aligned_TCA/1_make_consistent_fr_factors.py   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230515/aligned_TCA/1_make_consistent_fr_factors.py
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230515/aligned_TCA/dispatcher.py   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230515/aligned_TCA/dispatcher.py
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230515/aligned_TCA/notes.txt   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/2023051

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230528/jobNum_0/params.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230528/jobNum_0/params.json
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230528/jobNum_0/config.yaml   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230528/jobNum_0/config.yaml
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230528/jobNum_0/print_log_12021002.log   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230528/jobNum_0/print_log_12021002.log
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230528/jobNum_0/cpu_tracker.csv   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230528/jobNum_0/cpu_tracker.csv
downloading /n/data1/hms/neurobio/sabatini/rich/an

...

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230516/jobNum_0/analysis_files/VQT_Analyzer.h5   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230516/jobNum_0/analysis_files/VQT_Analyzer.h5
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230516/jobNum_0/analysis_files/Dataset_videos.h5   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230516/jobNum_0/analysis_files/Dataset_videos.h5
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230516/jobNum_0/pipeline_basic.py   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230516/jobNum_0/pipeline_basic.py
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230516/jobNum_0/run_info.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230516/jobNum_

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230509/jobNum_0/run_info.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230509/jobNum_0/run_info.json
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230509/jobNum_0/sbatch_config.sh   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230509/jobNum_0/sbatch_config.sh
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230509/aligned_TCA/parameters_batch.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230509/aligned_TCA/parameters_batch.json
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230509/aligned_TCA/jobNum_0/tca_refit.h5   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230509/aligned_TCA/jobNum_0/tca_refit.h5
downloading /n

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230529/aligned_TCA/dispatcher.py   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230529/aligned_TCA/dispatcher.py
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230529/aligned_TCA/notes.txt   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230529/aligned_TCA/notes.txt
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230529/pipeline_basic.py   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230529/pipeline_basic.py
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230529/dispatcher.py   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230529/dispatcher.py
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_2023

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230503/notes.txt   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230503/notes.txt
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230524/parameters_batch.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230524/parameters_batch.json
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230524/jobNum_0/params.json   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230524/jobNum_0/params.json
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230524/jobNum_0/config.yaml   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230524/jobNum_0/config.yaml
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/2023052

downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230502/jobNum_0/cpu_tracker.csv   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230502/jobNum_0/cpu_tracker.csv
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230502/jobNum_0/print_log_12079182.log   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230502/jobNum_0/print_log_12079182.log
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230502/jobNum_0/analysis_files/ROIs.h5   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230502/jobNum_0/analysis_files/ROIs.h5
downloading /n/data1/hms/neurobio/sabatini/rich/analysis/faceRhythm/mouse_0322N/run_20230701/20230502/jobNum_0/analysis_files/TCA.h5   to   /media/rich/bigSSD/analysis_data/face_rhythm/mouse_0322N/run_20230701/20230502/jobNum_0/analysis_files/TCA.h5
download

## Clean up

In [ ]:
## delete password
del pw
gc.collect()

## close sftp
if sftp: sftp.close()
if transport: transport.close()
del sftp
del transport
    
## close ssh
ssh_t.close()
del ssh_t
ssh_c.close()
del ssh_c
gc.collect()